<a href="https://colab.research.google.com/github/Location-Artistry/GEO-DEV-NOTEBOOKS/blob/main/agol_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creating a new class based on the ArcGIS Python API GIS object
### 01-29-2021
Previously working with large collection of functions without a defined class.   
This model made it difficult to keep track of what function were available and workflows.  Additionally it can get confusing whether you are working with a GIS object, item.id, item, or Feature Layer!    
AGOL CLASS based on Python API GIS() Object, with newly defined functions for easier access.  Working on creating an ITEM CLASS, which is obtained with the gis.conten.get(id) method.  ITEM can be any type of AGOL item that has an id: WebMap, Feature Class, StoryMap etc...   
Will need to shift some of the methods from AGOL to ITEM, will allow access to various information related to the items.  Looking for a method to list all of the Server url's for an entire organization/user?   
Also distinguish difference between an ITEM and a LAYER, and several layers can be contained within one ITEM/Feature Service

### UPDATE FROM WQAR 2-1-2021

In [ ]:
# Create new class from ArcGIS Python API functions
class agol:
  # -init- function, get user credentials to access GIS object, new Class = agol
  def __init__(self):
    #userID = input(f'ArcGIS Online USER ID: ')
    #pwd = getpass.getpass('PASSWORD: ')
    userID = 'LocationArtistry'
    pwd = 'pinecreek1'
    try:  
      self.gis = GIS("https://www.arcgis.com", userID, pwd)
      display(f'SUCCESS - CONNECTED TO: {self.gis.users.me.username} ACCOUNT as <gis>')
      display(self.gis)
    except:
        display(f'ERROR DID NOT CONNECT TO: {userID}')
# get list of all owner AGOL items, print list with title, id, type, and categories
  def getContent(self):
    try:
      my_content = self.gis.content.search(query="owner:" + self.gis.users.me.username, item_type="", max_items=200)
      for x in my_content:
        strMod = str(x.modified)
        stampInt = int(strMod[0:10])
        print(f'{x.title} - {x.id} - {x.type} - {x.categories} - {dt.datetime.fromtimestamp(stampInt)}')
      return my_content
    except:
      print('ERROR could not get user content')
# Clone item using id of item passed to function
  def cloneItem(self, gis2, cloneID):    
    try:
      itemToClone = self.gis.content.get(cloneID)
      print('Cloning:' + itemToClone.title + ' - ' + itemToClone.id + ' -',itemToClone.type)
      clonedItem = gis2.gis.content.clone_items(items=[itemToClone])
      print(f'Cloned Item: {clonedItem[0]}')
      #return clonedItem
    except:
      print('ERROR Could Not Clone')
# updated searchByKeywords, returns LIST of items 8-14-2020
  def search(self, keyWords):
    try:
        search = self.gis.content.search(query=f'{keyWords}', item_type='', max_items=50)
        for x, z in enumerate(search):
          strMod = str(z.modified)
          stampInt = int(strMod[0:10])
          print(f'{x} - {z.title} - {z.id} - {z.type} - {z.categories} - {dt.datetime.fromtimestamp(stampInt)}')
        return search
    except:
        print('ERROR Search not Successful')
# find item by keywords and display visual card
  def searchViz(self, keyWords):
    try:
        search = self.gis.content.search(query=f'{keyWords}', item_type='', max_items=50)
        for z in search:
            print(f'title: {z.title} - itemID: {z.id} - type: {z.type}')
            display(z)
    except:
        print('ERROR Search not Successful')
# return all keys and values for item when passed itemID string
  def itemInfo(self, id):
    try:
      getInfo = self.gis.content.get(id)
      for key, value in getInfo.items():
        display(f'{key}: {value}')
    except:
      display('ERROR GET Keys/Values not Successful')    
# takes itemID and gets and returns layerObject if exist, otherwise 'no layers found'
  def getLayers(self, id):
    try:
      feature = self.gis.content.get(id)
      featLay = feature.layers
      for z, x in enumerate(featLay):
        display(f'Layer {z}: {x}')
        return featLay
    except:
        display('no layers found')
# supply feature layer itemID, and the layer number to display table head
# Needs work, displays Feature Layer, not related table 
  def getLayerTable(self, id, layNum):
    try:
      layOut = self.getLayers(id)
      layQuery = layOut[layNum].query()
      display(layQuery.sdf.head())
    except:
      print('ERROR no Tables Found')
# delete item by itemID
  def deleteItem(self, id):
    delItem = self.gis.content.get(id)
    display(delItem)
    delQuest = input(f'Are you sure you want to delete: {delItem.title}')
    try:
      if delQuest.upper() == 'YES':
        display(f'DELETING: {delItem.title}')
        delItem.delete()
      else:
        display(f'NOT DELETING: {delItem.title}')
    except:
        display(f'ERROR failed to DELETE: {delItem.title}')
# create new agolItem class
  def getItem(self, id):
    try:
      item = agolItem(self, id)
      return item
    except:
        display(f'ERROR failed to GET item ID: {id}')

class agolItem:
# -init- function - create new agol item from agol CLASS and item.id
  def __init__(self,agol,id):
    try:  
      self.item =  agol.gis.content.get(id)
      display(self.item)
    except:
        display(f'ERROR failed to GET item ID: {id}')
  def info(self):
    try:
      keys, infoList = ['id', 'description', 'extent', 'spatialReference', 'url', 'owner', 'layers', 'tables'], []
      for i, z in enumerate(keys):
        infoList.append(self.item[z])
        display(f'{i} - {z} - {self.item[z]}')
      return infoList
    except:
      display(f'FAILED to get infoList for {self.item.title}')
# -init function - get layer from item create new agolLayer Class
  def getLayer(self):
    try:
      for i, lyr in enumerate(self.item.layers):
        display(f'{i} - {lyr.properties.name}')
      time.sleep(1)
      ind = int(input('Select index of Layer: '))
      url = self.item.url
      layer = agolLayer(url, ind)
      return layer
    except:
      display(f'Could not Get Layers for: {self.ags.url}')
  def getTable(self):
    try:
      for i, tab in enumerate(self.item.tables):
        display(f'{i} - {tab.properties.name}')
      time.sleep(1)
      ind = int(input('Select index of Layer: '))
      url = self.item.url
      layer = agolLayer(url, ind)
      return layer
    except:
      display(f'Could not Get Layers for: {self.ags.url}')
             
class agolLayer:
# -init function - get layer from item create new agolLayer Class 
  def __init__(self, url, ind):
    try:  
      layUrl = (f'{url}/{ind}')
      self.layer = FeatureLayer(layUrl)
    except:
        display(f'ERROR failed to GET Layer: {ind}')
  def layerQuery(self):
    try:
      baseURL = self.layer.url
      SQL = input(f'SQL query(ENTER for ALL): ') or '1=1'
      fields = input(f'Fields(ENTER for ALL)? ') or '*'
      url = (f'{baseURL}/query?f=geojson&where={SQL}&outFields={fields}')
      gdf = gpd.read_file(url)
      return gdf
    except:
      display(f'Query unsuccessful for: {self.item.title}')

class agolTable:
# -init function - get layer from item create new agolLayer Class 
  def __init__(self, url, ind):
    try:  
      tabUrl = (f'{url}/{ind}')
      self.table = FeatureLayer(tabUrl)
    except:
        display(f'ERROR failed to GET Layer: {ind}')
  def tableQuery(self):
    try:
      baseURL = self.table.url
      SQL = input(f'SQL query(ENTER for ALL): ') or '1=1'
      fields = input(f'Fields(ENTER for ALL)? ') or '*'
      url = (f'{baseURL}/query?f=geojson&where={SQL}&outFields={fields}')
      gdf = gpd.read_file(url)
      return gdf
    except:
      display(f'Query unsuccessful for: {self.item.title}')

## IMPORTS AND INSTALLS

In [ ]:
%%time 
import os
#os.environ['RESTAPI_USE_ARCPY'] = 'FALSE'
#os.environ['RESTAPI_VERIFY_CERT'] = 'FALSE'
!pip install bmi-arcgis-restapi
import restapi
#!apt update
#!apt upgrade
#!apt-get install -qq libgdal-dev libproj-dev

!apt install gdal-bin python-gdal python3-gdal
!apt install python3-rtree 
!pip install git+git://github.com/geopandas/geopandas.git
!pip install descartes
import geopandas as gpd
#!pip install contextily
#import contextily as ctx
!pip install pygeos
#!pip install folium
!apt-get install texlive-xetex texlive-fonts-recommended texlive-generic-recommended
import getpass

!pip install arcgis
from arcgis.gis import GIS, Item
#from arcgis.env import active_gis

from pathlib import Path
import sys
import os
import datetime as dt
import time as tm
import math
import getpass
from shapely.geometry import Point
#from IPython.display import display
from IPython.display import Markdown as md
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
#import folium
!pip uninstall tornado
!pip install tornado==4.5.3
# ArcGIS Python API libraries

#from arcgis.features import FeatureLayerCollection
#from arcgis.mapping import WebMap